# Flux and magnitude conversion

In this tutorial we will use the [F115W filter](http://svo2.cab.inta-csic.es/svo/theory/fps3/index.php?id=JWST/NIRCam.F115W&&mode=browse&gname=JWST&gname2=NIRCam#filter) of [JWST/NIRCam](https://www.jwst.nasa.gov/content/observatory/instruments/nircam.html) to convert a magnitude into a flux and vice versa. The magnitudes are defined in the Vega System with the magnitude of Vega set to 0.03 for each filter.

## Getting started

We start by importing `species` and initiating the database.

In [1]:
import species
species.SpeciesInit()

Initiating species v0.3.1... [DONE]
Creating species_config.ini... [DONE]
Database: /Users/tomasstolker/applications/species/docs/tutorials/species_database.hdf5
Data folder: /Users/tomasstolker/applications/species/docs/tutorials/data
Working folder: /Users/tomasstolker/applications/species/docs/tutorials
Creating species_database.hdf5... [DONE]
Creating data folder... [DONE]


## Magnitude to flux

We now create an instance of [SyntheticPhotometry](https://species.readthedocs.io/en/latest/species.analysis.html#species.analysis.photometry.SyntheticPhotometry) with the filter name as listed by the [SVO Filter Profile Service](http://svo2.cab.inta-csic.es/svo/theory/fps3/index.php?mode=browse&gname=JWST&gname2=NIRCam).

In [2]:
synphot = species.SyntheticPhotometry('JWST/NIRCam.F115W')

Adding filter: JWST/NIRCam.F115W... [DONE]


To covert from apparent magnitude to flux, we use the [magnitude_to_flux](https://species.readthedocs.io/en/latest/species.analysis.html#species.analysis.photometry.SyntheticPhotometry.magnitude_to_flux) method. The zero-point flux can be provided but is otherwise calculated from a flux-calibration Vega spectrum. Both the filter profile and the Vega spectrum will be downloaded and stored into the database. Let's calculate the flux for a magnitude of 15 +/- 0.2.

In [3]:
flux, error = synphot.magnitude_to_flux(15., error=0.2, zp_flux=None)
print(f'Flux (W m-2 um-1) = {flux:.2e} +/- {error:.2e}')

Adding Vega spectrum... [DONE]
Flux (W m-2 um-1) = 4.08e-15 +/- 7.56e-16


## Flux to magnitude

To convert backwards from flux to magnitude, we use the [flux_to_magnitude](https://species.readthedocs.io/en/latest/species.analysis.html#species.analysis.photometry.SyntheticPhotometry.flux_to_magnitude) method. When the `distance` is provide, both the apparent and absolute magnitude are calculated. The uncertainty on the distance is propagated into the uncertainty of the absolute magnitude

In [4]:
app_mag, abs_mag = synphot.flux_to_magnitude(flux, error=error, distance=(30., 5.))
print(f'Apparent magnitude = {app_mag[0]:.2f} +/- {app_mag[1]:.2f}')
print(f'Absolute magnitude = {abs_mag[0]:.2f} +/- {abs_mag[1]:.2f}')

Apparent magnitude = 15.00 +/- 0.20
Absolute magnitude = 12.61 +/- 0.42


As expected, we obtain again the magnitude that we started out with.

## Zero point flux

Finally, let's see what the zero-point flux (i.e. the flux of Vega) is for the `JWST/NIRCam.F115W` filter.

In [5]:
zp_flux = synphot.zero_point()
print(f'Zero point flux (W m-2 um-1) = {zp_flux:.2e}')

Zero point flux (W m-2 um-1) = 3.97e-09


This flux is indeed very similar to the zero point that is provided on the website of the [SVO Filter Profile Service](http://svo2.cab.inta-csic.es/svo/theory/fps3/index.php?id=JWST/NIRCam.F115W&&mode=browse&gname=JWST&gname2=NIRCam#filter): 3.993e-10 erg cm$^{-2}$ s$^{-1}$ A$^{-1}$.